In [64]:
import pandas as pd

In [65]:
# path to manifest file xslx
manifest_file = ""

# path to reference query reults
reference_file = ""

# output file name
dbml_filename = manifest_file.split('.')[0] + ".dbml"

In [66]:
db = "mssql"

In [67]:
# sheets in manifest excel workbook
is_sheet_name = "sistema"
tables_sheet_name = "lentelės"
columns_sheet_name = "stulpeliai"

In [68]:
df_is = pd.read_excel(manifest_file, sheet_name=is_sheet_name)
df_tables = pd.read_excel(manifest_file, sheet_name=tables_sheet_name)
df_columns = pd.read_excel(manifest_file, sheet_name=columns_sheet_name)

df_ref = pd.read_csv(reference_file, sep=';')

In [69]:
# replace $ and # with _ in table and column names
df_tables = df_tables.replace({'\$': '_', '#': '_'}, regex=True)
df_columns = df_columns.replace({'\$': '_', '#': '_'}, regex=True)
df_ref = df_ref.replace({'\$': '_', '#': '_'}, regex=True)

In [70]:
dbml_lines = []
tables = []
for row_index, row in df_tables.iterrows():
    table_name = row['table']
    tables.append(table_name)
    header = f"Table {table_name} {{\n"
    
    columns = ""
    for col_index, col in (df_columns[df_columns['table'] == row['table']]).iterrows():
        col_name = col['field']

        columns += f"\t{col_name} {(col['type']).replace(' ', '_')}"
        
        column_comment = str(col['comment']).replace("'", '"')
        column_note = f"note: '{column_comment}'"
        primary_key = "primary key" if col['is_primary'] else False
        
        if primary_key and (column_note != "note: 'nan'"):
            details = f"[{primary_key}, {column_note}]"
        elif primary_key:
            details = f"[{primary_key}]"
        elif (column_note != "note: 'nan'"):
            details = f"[{column_note}]"
        else:
            details = False

        if details:
            columns += f" {details}"
        
        columns += "\n"
    
    table_note = f"\tNote: '''\n\t\t{row['lenteles_paaiskinimas']}\n\t\tObjektas: {row['objektas']}\n\t\tĮrašu skaičius: {row['įrašų skaičius']} \n\t'''\n"
    footer = "}\n"
    dbml_line = header + columns + table_note + footer
    dbml_lines.append(dbml_line)

In [71]:
# mapping of column names
if db == "mssql":
    ref_cols = {
        "CHILD_TABLE": "tbl_child",
        "CHILD_COLUMN": "col_child",
        "CONSTRAINT_NAME": "const_name",
        "PARENT_TABLE": "tbl_parent",
        "PARENT_COLUMN": "col_parent"
    }

In [72]:
df_ref = df_ref.rename(columns=ref_cols)

In [73]:
for row_index, row in df_ref.iterrows():
    tbl_child = row["tbl_child"]
    col_child = row["col_child"]

    tbl_parent = row["tbl_parent"]
    col_parent = row["col_parent"]

    if tbl_child in tables and tbl_parent in tables:
        ref_line = f"Ref: {tbl_child}.{col_child} > {tbl_parent}.{col_parent} // {row['const_name']}\n"
        dbml_lines.append(ref_line)

In [74]:
with open(dbml_filename, 'w', encoding="utf8") as dmbl:
    dmbl.writelines(dbml_lines)